# Guia Rápido: Tratamento de Dados para Análise - Case NPS Algar Telecom

Bem-vindo! Vamos te mostrar de forma simples e direta como preparamos os dados do Case NPS Algar Telecom para uma análise mais eficiente no Power BI. Aqui está o passo a passo do que fizemos:

## Carregando os Dados

Primeiro, puxamos os dados do Excel. Sem mistério, apenas carregamos o arquivo para começar a trabalhar.

## Organizando com uma Chave Primária

Para facilitar a organização, criamos uma coluna `ID` que funciona como uma chave primária. Isso garante que cada linha seja única e bem identificada.

## Limpando o Texto

Alguns textos podem vir com problemas de codificação ou caracteres estranhos. Usamos uma ferramenta chamada `ftfy` para deixar tudo nos conformes. Isso foi aplicado em algumas colunas específicas que precisavam de um retoque.

## Formatando Datas

Datas são essenciais para análises temporais, então fizemos o seguinte:
- Convertido as datas para strings.
- Mantivemos apenas a parte da data (ano, mês e dia), ignorando a hora.
- Transformamos essas datas no formato `yyyy-mm-dd` para serem facilmente reconhecidas como datas no Power BI.

## Exportando o Resultado

Depois de tudo pronto, exportamos o DataFrame tratado para um arquivo CSV. Agora, ele está pronto para ser importado no Power BI e começar a análise.

Pronto! Agora seus dados estão organizados e preparados para uma análise poderosa no Power BI.


In [2]:
import pandas as pd
import ftfy
# import re

# Load the data from the Excel file
df = pd.read_excel('..\\data\\Case NPS - Algar Telecom.xlsx', sheet_name='Base dados')

# Add a primary key column called 'ID'
df['ID'] = range(1, len(df) + 1)

# Move the 'ID' column to be the first column in the DataFrame
columns = ['ID'] + [col for col in df.columns if col != 'ID']
df = df[columns]

# Check the text before applying fixes - this index was randomly selected
print('Product bad text example: ' + df['produto'][9])  # Display the original text in the 'produto' column at index 9
print('Origem voz bad text example: ' + df['origem_voz'][110178])  # Display the original text in the 'origem_voz' column at index 110178

# List of columns that need text correction
columns_to_fix_text = ['origem_voz', 'produto', 'regional', 'comentario', 'classificacao', 'classificacao_n2']

# Function to fix text issues using ftfy
def fix_text(df_series):
    return df_series.apply(lambda x: ftfy.fix_text(x) if isinstance(x, str) else x)

# Apply the text correction function to each specified column
for column in columns_to_fix_text:
    df[column] = fix_text(df[column])

# Check the text after applying fixes
print('Product fixed text example: ' + df['produto'][9])  # Display the corrected text in the 'produto' column at index 9
print('Origem voz fixed text example: ' + df['origem_voz'][110178])  # Display the corrected text in the 'origem_voz' column at index 110178

# Convert the 'data_voz' column to string to facilitate length filtering
df['data_voz'] = df['data_voz'].astype(str)

# Create a new column 'len_data_voz' to store the length of each string in 'data_voz'
df['len_data_voz'] = df['data_voz'].apply(len)

# Format 'data_voz' to 'yyyy-mm-dd' if it's 8 characters long and numeric
df['data_voz'] = df['data_voz'].apply(lambda x: f"{x[:4]}-{x[4:6]}-{x[6:]}" if len(x) == 8 and x.isdigit() else x)

# Remove any time component, keeping only the date
df['data_voz'] = df['data_voz'].str[:10]

# Convert 'data_voz' to datetime, handling errors by coercing invalid dates to NaT
df['data_voz'] = pd.to_datetime(df['data_voz'], errors='coerce')

# Count the number of NaT (Not a Time) values in 'data_voz'
num_NaT = df['data_voz'].isna().sum()
print(f"Errors Quantity after date transform: {num_NaT}")

# Export the cleaned DataFrame to a CSV file, using '|' as a separator
df.to_csv('export_telecom_case_formatted.csv', index=False, sep='|')


Product bad text example: BANDA LARGA EXP SOLUÃƒÂ‡ÃƒÂ•ES - CONVENCIONAL - 100 MB
Origem voz bad text example: Jornada AquisiÃ§Ã£o Varejo
Product fixed text example: BANDA LARGA EXP SOLUÇÕES - CONVENCIONAL - 100 MB
Origem voz fixed text example: Jornada Aquisição Varejo
Errors Quantity after date transform: 0
